# Object-oriented 'Point' class
## Introduction
We are developing a `Point` class capable of handling some simple linear algebra operations in 2D.

## Function `point_repr`

The first step in defining most classes is to define their `__init__` and `__repr__` methods so that we can construct and represent distinct objects of that class. Our `Point` class should accept two arguments, `x` and `y`, and be represented by a string `'Point(x, y)'` with appropriate values for `x` and `y`.

In [12]:
class Point(object):

    def __init__(self, x, y):
        self.x = x
        self.y = y
            
    def __repr__(self):
        return f"Point({self.x}, {self.y})"

## Function `add_substract`

The most basic vector operations we want our `Point` object to handle are addition and subtraction. For two points $(x_1, y_1) + (x_2, y_2) = (x_1 + x_2, y_1 + y_2)$ and similarly for subtraction. Implement a method within `Point` that allows two `Point` objects to be added together using the `+` operator, and likewise for subtraction.

In [4]:
class Point(object):

    def __init__(self, x, y):
        self.x = x
        self.y = y
            
    def __repr__(self):
        return f"Point({self.x}, {self.y})"
    
    def __add__(self, another):
        sum_x = self.x + another.x
        sum_y = self.y + another.y
        return Point(sum_x, sum_y)
    
    def __sub__(self, another):
        sub_x = self.x - another.x
        sub_y = self.y - another.y        
        return Point(sub_x, sub_y)

In [5]:
from functools import reduce
def add_sub_results(points):
    points = [Point(*point) for point in points]
    return [str(reduce(lambda x, y: x + y, points)), 
            str(reduce(lambda x, y: x - y, points))]

## Function `multiplication`

Within linear algebra there's many different kinds of multiplication: scalar multiplication, inner product, cross product, and matrix product. We're going to implement scalar multiplication and the inner product.

We can define scalar multiplication given a point $P$ and a scalar $a$ as 
$$aP=a(x,y)=(ax,ay)$$

and we can define the inner product for points $P,Q$ as
$$P\cdot Q=(x_1,y_1)\cdot (x_2, y_2) = x_1x_2 + y_1y_2$$


In [6]:
class Point(object):

    def __init__(self, x, y):
        self.x = x
        self.y = y
            
    def __repr__(self):
        return f"Point({self.x}, {self.y})"
    
    def __add__(self, another):
        sum_x = self.x + another.x
        sum_y = self.y + another.y
        return Point(sum_x, sum_y)
    
    def __sub__(self, another):
        sub_x = self.x - another.x
        sub_y = self.y - another.y        
        return Point(sub_x, sub_y)
    
    def __mul__(self, another):
        if isinstance(another, Point):
            return self.x*another.x + self.y*another.y
        if isinstance(another, (int, float, complex)):
            return Point(another*self.x, another*self.y)

In [7]:
def mult_result(points):
    points = [Point(*point) for point in points]
    return [point*point*2 for point in points]


## Function `Distance`

Another quantity we might want to compute is the distance between two points.  This is generally given for points $P_1=(x_1,y_1)$ and $P_2=(x_2,y_2)$ as 
$$D = |P_2 - P_1| = \sqrt{(x_1-x_2)^2 + (y_1-y_2)^2}.$$


In [8]:
from math import sqrt

class Point(object):

    def __init__(self, x, y):
        self.x = x
        self.y = y
            
    def __repr__(self):
        return f"Point({self.x}, {self.y})"
    
    def __add__(self, another):
        sum_x = self.x + another.x
        sum_y = self.y + another.y
        return Point(sum_x, sum_y)
    
    def __sub__(self, another):
        sub_x = self.x - another.x
        sub_y = self.y - another.y        
        return Point(sub_x, sub_y)
    
    def __mul__(self, another):
        if isinstance(another, Point):
            return self.x*another.x + self.y*another.y
        if isinstance(another, (int, float, complex)):
            return Point(another*self.x, another*self.y)
        
    def distance(self, another):
        return sqrt((self.x-another.x)**2 + (self.y-another.y)**2)

In [9]:
def dist_result(points):
    points = [Point(*point) for point in points]
    return [points[0].distance(point) for point in points]

## Algorithm implementation

Now we will use these points to solve a real world problem!  We can use our Point objects to represent measurements of two different quantities (e.g. a company's stock price and volume).  One thing we might want to do with a data set is to separate the points into groups of similar points.  Here we will implement an iterative algorithm to do this which will be a specific case of the very general $k$-means clustering algorithm.  The algorithm will require us to keep track of two clusters, each of which have a list of points and a center (which is another point, not necessarily one of the points we are clustering).  After making an initial guess at the center of the two clusters, $C_1$ and $C_2$, the steps proceed as follows

1. Assign each point to $C_1$ or $C_2$ based on whether the point is closer to the center of $C_1$ or $C_2$.
2. Recalculate the center of $C_1$ and $C_2$ based on the contained points. 

See [reference](https://en.wikipedia.org/wiki/K-means_clustering#Standard_algorithm) for more information.


In [35]:
class Cluster(object):
    def __init__(self, x, y):
        self.center = Point(x, y)
        self.points = []
    
    def update(self):
        if len(self.points) != 0:    
            x_center = sum([point.x for point in self.points])
            y_center = sum([point.y for point in self.points])
            self.center = Point(x_center/len(self.points), y_center/len(self.points))
    
    def add_point(self, point):
        self.points.append(point)

In [36]:
def compute_result(points):
    points = [Point(*point) for point in points]
    a = Cluster(1,0)
    b = Cluster(-1,0)
    a_old = []
    for _ in range(10000): # max iterations
        for point in points:
            if point.distance(a.center) < point.distance(b.center):
                a.add_point(point)
            else:
                b.add_point(point)
        if a_old == a.points:
            break
        a_old = a.points
        a.update()
        b.update()
        a.points = []
        b.points = []
    return sorted([(a.center.x, a.center.y), (b.center.x, b.center.y)], key= lambda x: x[0], reverse = True)

*Copyright &copy; 2020 The Data Incubator.  All rights reserved.*